In [ ]:
import openai
from tqdm.notebook import tqdm
import pandas as pd
import os
from IPython.display import display, Markdown
import pickle as pkl
pd.set_option('display.max_columns', 60)
import time

from datamining_utils import chatGPT_check_replay,chatGPT_replay,load_api_key_from_file,extract_code_script_from_markdown,find_dictionaries_in_string

%load_ext autoreload
%autoreload 2

In [ ]:

# Usage
api_key = load_api_key_from_file()


In [ ]:
from openai import OpenAI
client = OpenAI(api_key = api_key)

# Key Word Agent

In [ ]:
prop = 'LogD'

In [ ]:
assistant = client.beta.assistants.create(name='knowledge generation chatbot',
  instructions=f"Please summarize the ADME-T related important experimental conditions",
  model="gpt-4-1106-preview",
)   

In [ ]:
assistant.id

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Standardlise the experiments",
      "file_ids": assistant.file_ids
    }
  ]
)

In [ ]:
thread.id

### summarize the important experimental conditions

In [ ]:
mes = f"""
Please summarize the key experimental conditions for experiments related to {prop}.

Return a list of the distinct experimental condition names in Python list format.

Example output:
```python
['Experimental Condition A']
```
"""

In [ ]:
chatGPT_replay(thread.id, assistant, question_content=mes)

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)
display(Markdown(thread_messages.data[0].content[0].text.value))

In [ ]:
result = extract_code_script_from_markdown(thread_messages.data[0].content[0].text.value)[0]
experimental_conditions = eval(result)
experimental_conditions

In [ ]:
df = pd.read_csv('../data/raw_data/LogD/chembl_logd_raw_data.csv')


In [ ]:
condition = ' \n '.join(df['Assay Description'].value_counts().keys()[0:50])

mes = f"""
Please analyze the given assay descriptions and identify which experimental conditions are present within these descriptions.
Return a list of distinguishing experimental condition name can be found in the experimental conditions in Python list format.

{condition}
"""


In [ ]:
chatGPT_replay(thread.id, assistant, question_content=mes)

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)
display(Markdown(thread_messages.data[0].content[0].text.value))

In [ ]:
result = extract_code_script_from_markdown(thread_messages.data[0].content[0].text.value)[0]
result = result.split('=')[1]

experimental_conditions = eval(result)
experimental_conditions

In [ ]:
with open(f'../data/data_mining_results/{prop}_experimental_conditions_summaried_by_LLMs.pkl','wb') as f:
    pkl.dump(experimental_conditions,f)

# Examples Agent

In [ ]:
with open(f'../data/data_mining_results/{prop}/{prop}_experimental_conditions_summaried_by_LLMs.pkl','rb') as f:
    experimental_conditions = pkl.load(f)

In [ ]:
assistant = client.beta.assistants.create(name='Data Mining chatbot',
  instructions=f"Please mining the key biomedical information within the given data",
  model="gpt-4-1106-preview",
)   

In [ ]:
assistant.id

In [ ]:
mes = """
Biomedical data mining task.

Return python dictionary with key including index, original sentence, {}, and whether is {} experiment or not 
Fill in none if no information given. 
Please don't ignore some sentences.

""".format(', '.join(experimental_conditions),prop)

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": mes,
      "file_ids": assistant.file_ids
    }
  ]
)

In [ ]:
thread.id

In [ ]:
result = pd.DataFrame()
for i in tqdm(range(0,40,20)):
    info = ' \n '.join(df['Assay Description'].value_counts().keys()[i:i+20])
    
    chatGPT_replay(thread.id, assistant, question_content=info)
    time.sleep(3)
    runs = client.beta.threads.runs.list(
          thread.id)
    
    while runs.data[0].status != 'completed':
        time.sleep(3)
        runs = client.beta.threads.runs.list(
          thread.id
        )
    
    thread_messages = client.beta.threads.messages.list(thread.id)
    answer = extract_code_script_from_markdown(thread_messages.data[0].content[0].text.value)[0]
    if '=' in answer:
        answer = answer.split('=')[1]
    answer = eval(answer)
    

    display(pd.DataFrame(answer))
    result = pd.concat([result,pd.DataFrame(answer).T])

In [ ]:
result.to_csv(f'../data/data_mining_results/{prop}/example_{prop}.csv',index=False)

# Manul Validate and Create Full Promt with two shot Examples

In [ ]:
examples = pd.read_csv(f'../data/data_mining_results/{prop}/example_{prop}_manuel_validated.csv')

In [ ]:
tmp = examples[0:20]

' \n '.join(tmp[ 'original sentence'].values)

In [ ]:
str(tmp.to_dict(orient='list'))

In [ ]:
tmp = examples[20::]

' \n '.join(tmp[ 'original sentence'].values)

In [ ]:
str(tmp.to_dict(orient='list'))



f"""
Biomedical data mining task.

Return python dictionary with key including index, original sentence, {experimental_conditions_list}, and whether is {prop} experiment or not 
Fill in none if no information given. 
Please don't ignore some sentences.

Example 1 
Input: Lipophilicity, log D of the compound at pH 7.4 \n Lipophilicity, log D of the compound \n Partition coefficient (logD7.4) \n Lipophilicity, log D at pH 7.4 \n Distribution coefficient, log D of the compound \n SUPPLEMENTARY: Lipophilicity (log D) Determination by high-throughput shake-flask \n Lipophilicity, log D of the compound at pH 7.4 by HPLC analysis \n Lipophilicity, log D of the compound at pH 7.4 by chromatographic method \n Partition coefficient (logD6.5) \n Lipophilicity, log D of compound at pH 7.4 \n Lipophilicity, logD of the compound at pH 7.4 \n Distribution coefficient, log D of the compound at pH 7.4 \n Lipophilicity, log D of the compound at pH 7.4 by shake flask method \n Lipophilicity, logD of compound at pH 7.4 \n Lipophilicity, logD of compound \n Partition coefficient, log D at pH 7.4 \n Partition coefficient, log D of the compound at pH 7.4 by HPLC analysis \n Distribution coefficient, log D at pH 7.4 \n Distribution coefficient, logD of compound at pH 7.4 \n Partition coefficient (logD)

Output:
```python\n{{'index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'original sentence': ['Lipophilicity, log D of the compound at pH 7.4', 'Lipophilicity, log D of the compound', 'Partition coefficient (logD7.4)', 'Lipophilicity, log D at pH 7.4', 'Distribution coefficient, log D of the compound', 'SUPPLEMENTARY: Lipophilicity (log D) Determination by high-throughput shake-flask', 'Lipophilicity, log D of the compound at pH 7.4 by HPLC analysis', 'Lipophilicity, log D of the compound at pH 7.4 by chromatographic method', 'Partition coefficient (logD6.5)', 'Lipophilicity, log D of compound at pH 7.4', 'Lipophilicity, logD of the compound at pH 7.4', 'Distribution coefficient, log D of the compound at pH 7.4', 'Lipophilicity, log D of the compound at pH 7.4 by shake flask method', 'Lipophilicity, logD of compound at pH 7.4', 'Lipophilicity, logD of compound', 'Partition coefficient, log D at pH 7.4', 'Partition coefficient, log D of the compound at pH 7.4 by HPLC analysis', 'Distribution coefficient, log D at pH 7.4', 'Distribution coefficient, logD of compound at pH 7.4', 'Partition coefficient (logD)'], 'pH': [7.4, nan, 7.4, 7.4, nan, nan, 7.4, 7.4, 6.5, 7.4, 7.4, 7.4, 7.4, 7.4, nan, 7.4, 7.4, 7.4, 7.4, nan], 'Analytical Method': [nan, nan, nan, nan, nan, 'high-throughput shake-flask', 'HPLC', 'chromatography', nan, nan, nan, nan, 'shake flask', nan, nan, nan, 'HPLC', nan, nan, nan], 'Solvent System': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Equilibration Technique': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Incubation Time': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Shaking Condition': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'LogD experiment': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}}\n```

Example 2
Input: Lipophilicity, logD of the compound \n Lipophilicity, logD of compound at pH 7.4 by HPLC analysis \n DNDI: Lipophilicity (gLogD) at pH 7.4 \n DNDI: Lipophilicity (gLogD) at pH 3 \n Lipophilicity, log D of the compound by HPLC analysis \n Partition coefficient, log D of the compound at pH 7.4 \n Lipophilicity, log D of the compound by chromatography \n Partition coefficient (logD7.6) \n Lipophilicity, logD of compound at pH 7.4 by shake flask method \n Octanol-water partition coefficient, log D of the compound at pH 7.4 by shake flask method \n Chromatographic lipophilicity, log D of compound at pH 7.4 by UV-HPLC analysis \n Lipophilicity, log D of compound \n Buffer/octanol partition coefficient, log D of the compound at pH 7.4 by shake flask method \n DNDI: Lipophilicity measured in Chromatographic hydrophobicity index (CHI) assay, pH 7.4 \n Lipophilicity, logD of the compound at pH 7.4 by chromatographic method \n Octanol-water partition coefficient, log D of the compound at pH 7.4 \n Octanol-phosphate buffer partition coefficient, log D of the compound at pH 7.4 by shake flask method \n Lipophilicity, logD of the compound at pH 7.4 by shake flask method \n Lipophilicity, logD of compound at pH 7.4 by Chromatography \n Distribution coefficient, logD of the compound at pH 7.4

Output:
```python\n{{'index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'original sentence': ['Lipophilicity, logD of the compound', 'Lipophilicity, logD of compound at pH 7.4 by HPLC analysis', 'DNDI: Lipophilicity (gLogD) at pH 7.4', 'DNDI: Lipophilicity (gLogD) at pH 3', 'Lipophilicity, log D of the compound by HPLC analysis', 'Partition coefficient, log D of the compound at pH 7.4', 'Lipophilicity, log D of the compound by chromatography', 'Partition coefficient (logD7.6)', 'Lipophilicity, logD of compound at pH 7.4 by shake flask method', 'Octanol-water partition coefficient, log D of the compound at pH 7.4 by shake flask method', 'Chromatographic lipophilicity, log D of compound at pH 7.4 by UV-HPLC analysis', 'Lipophilicity, log D of compound', 'Buffer/octanol partition coefficient, log D of the compound at pH 7.4 by shake flask method', 'DNDI: Lipophilicity measured in Chromatographic hydrophobicity index (CHI) assay, pH 7.4', 'Lipophilicity, logD of the compound at pH 7.4 by chromatographic method', 'Octanol-water partition coefficient, log D of the compound at pH 7.4', 'Octanol-phosphate buffer partition coefficient, log D of the compound at pH 7.4 by shake flask method', 'Lipophilicity, logD of the compound at pH 7.4 by shake flask method', 'Lipophilicity, logD of compound at pH 7.4 by Chromatography', 'Distribution coefficient, logD of the compound at pH 7.4'], 'pH': [nan, 7.4, 7.4, 3.0, nan, 7.4, nan, 7.6, 7.4, 7.4, 7.4, nan, 7.4, 7.4, 7.4, 7.4, 7.4, 7.4, 7.4, 7.4], 'Analytical Method': [nan, 'HPLC', nan, nan, 'HPLC', nan, 'chromatography', nan, 'shake flask', 'shake flask', 'UV-HPLC', nan, 'shake flask', 'CHI assay', 'chromatography', nan, 'shake flask', 'shake flask', 'Chromatography', nan], 'Solvent System': [nan, nan, nan, nan, nan, nan, nan, nan, nan, 'octanol-water', nan, nan, 'buffer/octanol', nan, nan, 'octanol-water', 'octanol-phosphate buffer', nan, nan, nan], 'Equilibration Technique': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Incubation Time': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Shaking Condition': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'LogD experiment': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}}\n```

"""

# Data Mining Agent

### Load data 

In [ ]:
df = pd.read_csv('../data/raw_data/LogD/chembl_logd_raw_data.csv')
assay_description = df['Assay Description'].value_counts().keys()

### Create agent

In [ ]:
assistant = client.beta.assistants.create(name='Data Mining chatbot',
  instructions=f"Please mining the key biomedical information within the given data",
  model="gpt-4-1106-preview",
)   

In [ ]:
assistant.id

In [ ]:
experimental_conditions_list = ['pH',
 'Analytical Method',
 'Solvent System',
 'Equilibration Technique',
 'Incubation Time',
 'Shaking Condition']

experimental_conditions_list = ', '.join(experimental_conditions_list)

In [ ]:
mes = f"""
Biomedical data mining task.

Return python dictionary with key including index, original sentence, {experimental_conditions_list}, and whether is {prop} experiment or not 
Fill in none if no information given. 
Please don't ignore some sentences.

Example 1 
Input: Lipophilicity, log D of the compound at pH 7.4 \n Lipophilicity, log D of the compound \n Partition coefficient (logD7.4) \n Lipophilicity, log D at pH 7.4 \n Distribution coefficient, log D of the compound \n SUPPLEMENTARY: Lipophilicity (log D) Determination by high-throughput shake-flask \n Lipophilicity, log D of the compound at pH 7.4 by HPLC analysis \n Lipophilicity, log D of the compound at pH 7.4 by chromatographic method \n Partition coefficient (logD6.5) \n Lipophilicity, log D of compound at pH 7.4 \n Lipophilicity, logD of the compound at pH 7.4 \n Distribution coefficient, log D of the compound at pH 7.4 \n Lipophilicity, log D of the compound at pH 7.4 by shake flask method \n Lipophilicity, logD of compound at pH 7.4 \n Lipophilicity, logD of compound \n Partition coefficient, log D at pH 7.4 \n Partition coefficient, log D of the compound at pH 7.4 by HPLC analysis \n Distribution coefficient, log D at pH 7.4 \n Distribution coefficient, logD of compound at pH 7.4 \n Partition coefficient (logD)

Output:
```python\n{{'index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'original sentence': ['Lipophilicity, log D of the compound at pH 7.4', 'Lipophilicity, log D of the compound', 'Partition coefficient (logD7.4)', 'Lipophilicity, log D at pH 7.4', 'Distribution coefficient, log D of the compound', 'SUPPLEMENTARY: Lipophilicity (log D) Determination by high-throughput shake-flask', 'Lipophilicity, log D of the compound at pH 7.4 by HPLC analysis', 'Lipophilicity, log D of the compound at pH 7.4 by chromatographic method', 'Partition coefficient (logD6.5)', 'Lipophilicity, log D of compound at pH 7.4', 'Lipophilicity, logD of the compound at pH 7.4', 'Distribution coefficient, log D of the compound at pH 7.4', 'Lipophilicity, log D of the compound at pH 7.4 by shake flask method', 'Lipophilicity, logD of compound at pH 7.4', 'Lipophilicity, logD of compound', 'Partition coefficient, log D at pH 7.4', 'Partition coefficient, log D of the compound at pH 7.4 by HPLC analysis', 'Distribution coefficient, log D at pH 7.4', 'Distribution coefficient, logD of compound at pH 7.4', 'Partition coefficient (logD)'], 'pH': [7.4, nan, 7.4, 7.4, nan, nan, 7.4, 7.4, 6.5, 7.4, 7.4, 7.4, 7.4, 7.4, nan, 7.4, 7.4, 7.4, 7.4, nan], 'Analytical Method': [nan, nan, nan, nan, nan, 'high-throughput shake-flask', 'HPLC', 'chromatography', nan, nan, nan, nan, 'shake flask', nan, nan, nan, 'HPLC', nan, nan, nan], 'Solvent System': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Equilibration Technique': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Incubation Time': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Shaking Condition': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'LogD experiment': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}}\n```

Example 2
Input: Lipophilicity, logD of the compound \n Lipophilicity, logD of compound at pH 7.4 by HPLC analysis \n DNDI: Lipophilicity (gLogD) at pH 7.4 \n DNDI: Lipophilicity (gLogD) at pH 3 \n Lipophilicity, log D of the compound by HPLC analysis \n Partition coefficient, log D of the compound at pH 7.4 \n Lipophilicity, log D of the compound by chromatography \n Partition coefficient (logD7.6) \n Lipophilicity, logD of compound at pH 7.4 by shake flask method \n Octanol-water partition coefficient, log D of the compound at pH 7.4 by shake flask method \n Chromatographic lipophilicity, log D of compound at pH 7.4 by UV-HPLC analysis \n Lipophilicity, log D of compound \n Buffer/octanol partition coefficient, log D of the compound at pH 7.4 by shake flask method \n DNDI: Lipophilicity measured in Chromatographic hydrophobicity index (CHI) assay, pH 7.4 \n Lipophilicity, logD of the compound at pH 7.4 by chromatographic method \n Octanol-water partition coefficient, log D of the compound at pH 7.4 \n Octanol-phosphate buffer partition coefficient, log D of the compound at pH 7.4 by shake flask method \n Lipophilicity, logD of the compound at pH 7.4 by shake flask method \n Lipophilicity, logD of compound at pH 7.4 by Chromatography \n Distribution coefficient, logD of the compound at pH 7.4

Output:
```python\n{{'index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'original sentence': ['Lipophilicity, logD of the compound', 'Lipophilicity, logD of compound at pH 7.4 by HPLC analysis', 'DNDI: Lipophilicity (gLogD) at pH 7.4', 'DNDI: Lipophilicity (gLogD) at pH 3', 'Lipophilicity, log D of the compound by HPLC analysis', 'Partition coefficient, log D of the compound at pH 7.4', 'Lipophilicity, log D of the compound by chromatography', 'Partition coefficient (logD7.6)', 'Lipophilicity, logD of compound at pH 7.4 by shake flask method', 'Octanol-water partition coefficient, log D of the compound at pH 7.4 by shake flask method', 'Chromatographic lipophilicity, log D of compound at pH 7.4 by UV-HPLC analysis', 'Lipophilicity, log D of compound', 'Buffer/octanol partition coefficient, log D of the compound at pH 7.4 by shake flask method', 'DNDI: Lipophilicity measured in Chromatographic hydrophobicity index (CHI) assay, pH 7.4', 'Lipophilicity, logD of the compound at pH 7.4 by chromatographic method', 'Octanol-water partition coefficient, log D of the compound at pH 7.4', 'Octanol-phosphate buffer partition coefficient, log D of the compound at pH 7.4 by shake flask method', 'Lipophilicity, logD of the compound at pH 7.4 by shake flask method', 'Lipophilicity, logD of compound at pH 7.4 by Chromatography', 'Distribution coefficient, logD of the compound at pH 7.4'], 'pH': [nan, 7.4, 7.4, 3.0, nan, 7.4, nan, 7.6, 7.4, 7.4, 7.4, nan, 7.4, 7.4, 7.4, 7.4, 7.4, 7.4, 7.4, 7.4], 'Analytical Method': [nan, 'HPLC', nan, nan, 'HPLC', nan, 'chromatography', nan, 'shake flask', 'shake flask', 'UV-HPLC', nan, 'shake flask', 'CHI assay', 'chromatography', nan, 'shake flask', 'shake flask', 'Chromatography', nan], 'Solvent System': [nan, nan, nan, nan, nan, nan, nan, nan, nan, 'octanol-water', nan, nan, 'buffer/octanol', nan, nan, 'octanol-water', 'octanol-phosphate buffer', nan, nan, nan], 'Equilibration Technique': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Incubation Time': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Shaking Condition': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'LogD experiment': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]}}\n```

"""

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": mes,
      "file_ids": assistant.file_ids
    }
  ]
)

In [ ]:
thread.id

In [ ]:
len(assay_description)

In [ ]:
for i in tqdm(range(0,len(assay_description),20)):
    info = ' \n '.join(assay_description[i:i+20])
    
    chatGPT_replay(thread.id, assistant, question_content=info)
    time.sleep(3)
    runs = client.beta.threads.runs.list(
          thread.id)
    
    while runs.data[0].status != 'completed':
        time.sleep(3)
        runs = client.beta.threads.runs.list(
          thread.id
        )
    
    thread_messages = client.beta.threads.messages.list(thread.id)

    try:
        answer = extract_code_script_from_markdown(thread_messages.data[0].content[0].text.value)[0]
        if '=' in answer:
            answer = answer.split('=')[1]
        answer = eval(answer)
        pd.DataFrame(answer).to_csv(f'../data/data_mining_results/logd/batch/{prop}_batch_{i}.csv',index=False)
    except:
        print(f'error for {i}')
        display(thread_messages.data[0].content[0].text.value)
        continue


# Combine result

In [ ]:
import os
root_path = '../data/data_mining_results/LogD/batch/'
result = pd.DataFrame()
for file in os.listdir(root_path):
    if 'csv' in file:
        file_path = os.path.join(root_path,file)
        tmp = pd.read_csv(file_path)
        result = pd.concat([result,tmp])

In [ ]:
result.columns

In [ ]:
result = result[[ 'original sentence', 'pH', 'Analytical Method',
       'Solvent System', 'Equilibration Technique', 'Incubation Time',
       'Shaking Condition', 'LogD experiment']]

In [ ]:
result.columns = ['Assay Description','pH', 'Analytical Method',
       'Solvent System', 'Equilibration Technique', 'Incubation Time',
       'Shaking Condition', 'LogD experiment']

In [ ]:
df = pd.read_csv('../data/raw_data/LogD/chembl_logd_raw_data.csv')
col_list = ['Molecule ChEMBL ID', 
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 
       'Document ChEMBL ID', 'Source Description',
       'Document Journal', 'Document Year']
df = df[col_list]

In [ ]:
df = df.merge(result,on='Assay Description',how='left')
df

In [ ]:
df.to_csv('../data/data_mining_results/LogD/LogD_Chembl_data_mining_finished.csv',index=False)